In [1]:
# ingestion.py

import getpass
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import pickle
from langchain.retrievers import BM25Retriever,EnsembleRetriever
from kiwipiepy import Kiwi
import cohere
import dill

load_dotenv(override=True) # 강제 다시 로드

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

if not os.environ.get("PINECONE_API_KEY"):
  os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter Pinecone API key: ")

pinecone_api = os.environ["PINECONE_API_KEY"]
cohere_api = os.environ["COHERE_API_KEY"]

# vectorstore load
pc = Pinecone(api_key=pinecone_api)

index_name = "canon"
index = pc.Index(index_name)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = PineconeVectorStore(embedding=embeddings, index=index)


retriever = vector_store.as_retriever(
  search_type="similarity", search_kwargs={"k": 10},
)

# 새로 초기화된 Kiwi와 동일한 토크나이징 함수 적용
kiwi = Kiwi()
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

# === 1. BM25Retriever와 Kiwi 로드 ===
with open("../chunk_result/bm25_retriever_r50.pkl", "rb") as f:
    bm25_retriever = dill.load(f)


bm25_retriever.preprocess_func = kiwi_tokenize

# === 3. Ensemble Retriever 생성 ===
ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm25_retriever],
    weights=[0.5, 0.5]  # Dense와 BM25 각각 50% 가중치
)

cohere_client = cohere.Client(cohere_api)

In [2]:
# state.py
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

# class ContextState(TypedDict):
#     ensemble_context: Annotated[str, "Ensemble Retrieve"]
#     transform_question: Annotated[list, "Transformed queries generated by LLM"]
#     multi_context: Annotated[str, "Multi Query"]

# # class MultiQueryState(TypedDict):

# GraphState 상태 정의
class GraphState(TypedDict):
    question: Annotated[str, "Question"]
    transform_question: Annotated[list, "Transformed queries generated by LLM"]
    ensemble_context: Annotated[str, "Ensemble Retrieve"]
    multi_context: Annotated[str, "Multi Query"]
    merge_context: Annotated[str, "Merge Context"]
    rerank_context : Annotated[str, "Context"]
    answer: Annotated[str, "Answer"]
    message: Annotated[list, add_messages]

In [3]:
# retrieve.py
# from state import GraphState
# from ingestion import retriever

# ensemble retriever 로 변경
def retrieve_document(state: GraphState) -> GraphState:
    print("---RETRIEVE---\n")
    questions = state["question"]
    documents = ensemble_retriever.invoke(questions)
    print(documents)
    return {"ensemble_context": documents}

In [5]:
# multiquery.py
from typing import List
from pydantic import BaseModel, Field
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage
from dotenv import load_dotenv
import json

load_dotenv()

# output 정의
class LineListOutputParser(BaseOutputParser[List[str]]):
    def parse(self, text: str) -> List[str]:
        if isinstance(text, AIMessage):
            text = text.content
        
        try:
            parsed_json = json.loads(text)
            return {"lines": parsed_json}
        except:
            lines = text.strip().split("\n")
            return {"lines":list(filter(None, lines))}
        # return list(filter(None, lines))  # Remove empty lines

output_parser = LineListOutputParser()


QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions in a JSON array format, separated by commas.
    Do not include any additional explanations.
    Original question: {question}
    Output format: ["question1", "question2", "question3", "question4", "question5"]""",
)

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

llm_chain = QUERY_PROMPT | llm | output_parser

# Node for generating transformed queries
def generate_transformed_queries(state: GraphState) -> GraphState:
    print("---QUERY GENERTATE---")
    query = state["question"]
    transformed_queries = llm_chain.invoke({'question':query})
    print(transformed_queries)
    return {"transform_question": transformed_queries}

In [6]:
# multiQuery retreiver.py
# from ingestion import retriever
# from state import GraphState
# from multiquery import llm_chain
from langchain.retrievers.multi_query import MultiQueryRetriever

# Initialize the retriever
multiquery_retriever = MultiQueryRetriever(
    retriever=retriever, llm_chain=llm_chain, parser_key="lines"
)

# Node for retrieving documents
def multiquery_retrieve(state: GraphState) -> GraphState:
    print("---QUERY RETRIEVE---")
    transformed_queries = state["transform_question"]
    print(transformed_queries)
    unique_docs = multiquery_retriever.invoke(transformed_queries)
    print(unique_docs)
    return {"multi_context": unique_docs}

In [7]:
# query_merge.py

def merge_results(state: GraphState) -> GraphState:
    print("---MERGE---")

    multi_query_result = state['multi_context']
    ensemble_result = state['ensemble_context']
    print(multi_query_result)
    print(ensemble_result)

    # 중복 제거 (예: 문서 ID 기준)
    seen_ids = set()
    merged_result = []

    for item in multi_query_result + ensemble_result:
        if item.id not in seen_ids:
            merged_result.append(item)
            seen_ids.add(item.id)
    print(state)

    return {'merge_context': merged_result}


In [8]:
# rerank.py

def rerank_with_cohere(query, retrieved_docs, top_n=5):
    # Cohere에 전달할 문서 형식
    documents = [doc.page_content for doc in retrieved_docs]
    
    # Reranker 호출
    response = cohere_client.rerank(
        query=query,
        documents=documents,
        top_n=top_n,  # 상위 N개 문서 선택
        model="rerank-v3.5"  # 사용할 Cohere Reranker 모델
    )
    
    # 상위 문서만 반환
    reranked_docs = [retrieved_docs[result.index] for result in response.results]
    return reranked_docs

# Reranker Node
def rerank_docs(state: GraphState) -> GraphState:
    print("---RERANK---")
    print(state)
    questions = state['question']
    documents = state['merge_context']
    reranked_docs = rerank_with_cohere(questions, documents)
    return {"rerank_context": reranked_docs}

In [9]:
# generation.py
# 답변 생성 체인
from dotenv import load_dotenv
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(temperature=0)
prompt = hub.pull("rlm/rag-prompt")

generation_chain = prompt | llm | StrOutputParser()

/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/.venv/lib/python3.10/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [10]:
# generate.py
# 답변 실행 역할

from typing import Any, Dict

# from generation import generation_chain
# from state import GraphState

def generate(state: GraphState) -> Dict[str, Any]:
    print("---GENERATE---")
    question = state["question"]
    context = state["rerank_context"]

    generation = generation_chain.invoke({"context": context, "question": question})

    message = [{"role": "user", "content": question},{"role":"assistant", "content":generation}]
    return {"question": question, "answer": generation, "message": message}

In [11]:
# consts.py
RETRIEVE = "retrieve"
GRADE_DOCUMENTS = "grade_documents"
GENERATE = "generate"
WEBSEARCH = "websearch"

# graph.py
from dotenv import load_dotenv
from langgraph.graph import START, END, StateGraph
from langgraph.types import Send
# from langchain.schema.runnable import RunnableParallel


# from consts import RETRIEVE, GRADE_DOCUMENTS, GENERATE, WEBSEARCH

# from nodes import generate, grade_documents, retrieve, web_search
# from state import GraphState

load_dotenv()

workflow = StateGraph(GraphState)


workflow.add_node("ensemble retrieve", retrieve_document)
workflow.add_node("multi query generate", generate_transformed_queries)
workflow.add_node("multi query retrieve", multiquery_retrieve)
workflow.add_node("merge retrieve", merge_results)
workflow.add_node("rerank", rerank_docs)
workflow.add_node("generate", generate)

# workflow.set_entry_point("retrieve")
workflow.add_edge(START, 'ensemble retrieve')
workflow.add_edge(START, 'multi query generate')
workflow.add_edge('multi query generate', 'multi query retrieve')

# 조건부 전환 함수 정의
def check_conditions(state: GraphState):
    # 두 결과가 모두 존재하는지 확인
    if "ensemble_context" in state and "multi_context" in state:
        return [Send("merge retrieve", state)]  # 조건 충족 시 다음 노드로 전달
    return []  # 조건 미충족 시 빈 리스트 반환 (대기 상태)

# 조건부 전환 추가
workflow.add_conditional_edges(
    "ensemble retrieve",
    check_conditions,
    ["merge retrieve"]
)

workflow.add_conditional_edges(
    "multi query retrieve",
    check_conditions,
    ["merge retrieve"]
)


workflow.add_edge("merge retrieve", "rerank")
workflow.add_edge("rerank", "generate")
workflow.add_edge("generate", END)

app = workflow.compile()

In [12]:
app.get_graph().draw_mermaid_png(output_file_path='./graph_node_0202.png')

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x95\x00\x00\x02v\x08\x02\x00\x00\x00\x18\x01\x88\xdd\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00 \x00IDATx\x9c\xec\xddw\\S\xd7\xc3\x06\xf0\x132\xd8{\x0f\x11T\x9c\xb8qO\xc4\x89,\x11\xd1"\xce\xdaV\xc5\xbd\xab\xb6\xb5\xee:\xaau\xb7\xee\xbdQ\x94\xaa\xb8\x07\xee\x85{\x147 {Cv\xf2\xfeq})?66pr\x93\xe7\xfb\x87\x9f\x90\xdc\xdc<1\x97\x87\x93\x9b\x9bs9J\xa5\x92\x00\x00\xb0\x90\x0e\xed\x00\x00\x00_\t\xfd\x05\x00l\x85\xfe\x02\x00\xb6B\x7f\x01\x00[\xa1\xbf\x00\x80\xad\xd0_\x00\xc0V<\xda\x01\xa0Z\xa5\xc6\x8b\xf2\xb2\xe5\xf99r\x89H!\x16*h\xc7\xa9\x10\x81\x9e\x0e\x8f\xc710\xe1\x1a\x18s\xed\\\xf4i\xc7\x015\xc2\xc1\xf1_\xda\xe0\xe3\xcb\xfc\xb7Or\xdf>\xcdsr\xd3\x17\xe5)\x0c\x8c\xb9f\xd6\x02\x85\x9c\x1d/\xbd@_\'#Y\x92\x9f-W*\x95\x1f\x9e\xe7\xbb\xba\x1b\xba\xba\x1b6hmB;\x17\xd0\x87\xfe\xd2p\x9f^\xe7\xdf8\x91f\xe5(\xb0q\xd6\xab\xe5nhh\xca\xee\x11\xb7B\xa1|\xf74\xef\xdd\xd3\xbc\xf7\xcf\xf3Z\xf5\xb4h\xda\xd9\x8cv"\xa0\t\xfd\xa5\xc9\xce\xefK\xca\xcd\x94\xb5\

In [13]:
print(app.invoke(input={"question": "iso 설정 방법에 대해 알려줘"}))

---QUERY GENERTATE------RETRIEVE---


{'lines': ['iso 설정하는 방법은 무엇인가요?', 'iso 설정 절차에 대해 설명해 주세요.', 'iso 설정을 위한 단계는 어떻게 되나요?', 'iso 설정 방법에 대한 정보를 알고 싶습니다.', 'iso 설정을 위한 가이드를 제공해 주실 수 있나요?']}
[Document(id='chunk-r50-680', metadata={'brand': 'Canon', 'image_path': ['/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_664_1.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_664_2.png'], 'index': '탭 메뉴: 설정', 'main_index': '설정', 'model': 'R50', 'page': 664.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_664.jpg', 'sub_index': 'N/A'}, page_content='## 설정 3\n\n![그림 자리(설정 3 메뉴 화면)]\n\n1. **화면/뷰파인더 표시**\n2. **화면 밝기**\n3. **뷰파인더 밝기**\n4. **뷰파인더 색조 미세조정**\n5. **메뉴 화면 확대**\n6. **HDMI 해상도**\n\n## 설정 4\n\n![그림 자리(설정 4 메뉴 화면)]\n\n1. **터치 제어**\n2. **USB 연결 앱 선택**'), Document(i

In [14]:
inputs = {"question": "iso 설정 방법에 대해 알려줘"}

for chunk_msg, metadata in app.stream(inputs, stream_mode="messages"):
        print(chunk_msg.content, end="", flush=True)

---RETRIEVE---

---QUERY GENERTATE---
["iso 설정하는 방법을 설명해줘", "iso 설정 절차에 대해 알고 싶어", "iso 설정을 어떻게 하는지 알려줄 수 있어?", "iso 설정 방법에 대한 정보를 제공해줘", "iso 설정 관련 팁이나{'lines': ['iso 설정하는 방법을 설명해줘', 'iso 설정 절차에 대해 알고 싶어', 'iso 설정을 어떻게 하는지 알려줄 수 있어?', 'iso 설정 방법에 대한 정보를 제공해줘', 'iso 설정 관련 팁이나 가이드를 줄 수 있니?']}
 가이드를 줄 수 있니?"][Document(id='chunk-r50-680', metadata={'brand': 'Canon', 'image_path': ['/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_664_1.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_664_2.png'], 'index': '탭 메뉴: 설정', 'main_index': '설정', 'model': 'R50', 'page': 664.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_664.jpg', 'sub_index': 'N/A'}, page_content='## 설정 3\n\n![그림 자리(설정 3 메뉴 화면)]\n\n1. **화면/뷰파인더 표시**\n2. **화면 밝기**\n3. **뷰파인더 밝기**\n4. **뷰파인더 색조 미세조정**

In [21]:
from langchain.docstore.document import Document

retrieveal_doc = [Document(id='chunk-680', metadata={'brand': 'Canon', 'image_path': ['/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_664_1.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_664_2.png'], 'index': '탭 메뉴: 설정', 'main_index': '설정', 'model': 'R50', 'page': 664.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_664.jpg', 'sub_index': 'N/A'}, page_content='## 설정 3\n\n![그림 자리(설정 3 메뉴 화면)]\n\n1. **화면/뷰파인더 표시**\n2. **화면 밝기**\n3. **뷰파인더 밝기**\n4. **뷰파인더 색조 미세조정**\n5. **메뉴 화면 확대**\n6. **HDMI 해상도**\n\n## 설정 4\n\n![그림 자리(설정 4 메뉴 화면)]\n\n1. **터치 제어**\n2. **USB 연결 앱 선택**'), Document(id='chunk-678', metadata={'brand': 'Canon', 'image_path': [], 'index': 'N/A', 'main_index': '설정', 'model': 'R50', 'page': 662.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_662.jpg', 'sub_index': 'N/A'}, page_content='# 설정\n\n이 장에서는 설정 탭의 메뉴 설정에 관해 설명합니다.  \n제목 우측의 ★는 해당 기능이 크리에이티브 존 모드 (<P>, <Tv>, <Av>, <M>)에서만 사용 가능함을 나타냅니다.\n\n- 탭 메뉴: 설정\n- 폴더 설정\n- 파일 번호\n- 카드 포맷\n- 자동 회전\n- 동영상에 방향 정보 추가하기\n- 날짜/시간/지역\n- 언어\n- 비디오 형식\n- 촬영 모드 안내\n- 기능 안내\n- 표시음\n- 음량\n- 전원\n- 스크린 및 뷰파인더 표시\n- 스크린 밝기\n- 뷰파인더 밝기\n- 뷰파인더 색조 미세 조정\n- 메뉴 화면 확대\n- HDMI 해상도\n- 터치 제어\n- USB 연결을 위한 앱 선택\n- 카메라 설정 초기화 ★\n- 커스텀 촬영 모드 (C 모드)\n- 배터리 정보\n- 저작권 정보 ★\n- 기타 정보'), Document(id='chunk-236', metadata={'brand': 'Canon', 'image_path': ['/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_236_1.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_236_2.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_236_3.png'], 'index': '픽쳐 스타일 사용자 설정', 'main_index': '촬영 및 녹화', 'model': 'R50', 'page': 236.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_236.jpg', 'sub_index': '정지 사진 촬영'}, page_content='### 3. 옵션을 선택합니다.\n\n![옵션 선택 화면]\n\n- 옵션을 선택한 다음 <▶> 버튼을 누르십시오.\n- 설정과 효과에 관한 자세한 내용은 설정과 효과를 참조하십시오.\n\n### 4. 효과 레벨을 설정합니다.\n\n![효과 레벨 설정 화면]\n\n- 효과 레벨을 조정한 다음 <▶> 버튼을 누르십시오.\n- `<MENU>` 버튼을 누르면 조정한 설정값이 저장되고 픽쳐 스타일 선택 화면으로 돌아갑니다.\n- 초기 설정에서 변경된 설정값은 청색으로 표시됩니다.'), Document(id='chunk-240', metadata={'brand': 'Canon', 'image_path': ['/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_240_1.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_240_2.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_240_3.png'], 'index': '픽쳐 스타일 등록', 'main_index': '촬영 및 녹화', 'model': 'R50', 'page': 240.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_240.jpg', 'sub_index': '정지 사진 촬영'}, page_content='### 3. <SET> 버튼을 누릅니다.\n\n![그림 자리(픽처스타일 설정 화면)]\n\n- [픽처스타일]을 선택한 다음 <SET> 버튼을 누르십시오.\n\n### 4. 기본 픽처 스타일을 선택합니다.\n\n![그림 자리(기본 픽처 스타일 선택 화면)]\n\n- 기본 픽처 스타일을 선택한 다음 <SET> 버튼을 누르십시오.\n- EOS Utility (EOS 소프트웨어)로 카메라에 등록한 스타일을 조정할 때도 이 방법으로 스타일을 선택하십시오.\n\n### 5. 옵션을 선택합니다.\n\n![그림 자리(옵션 선택 화면)]\n\n- 옵션을 선택한 다음 <SET> 버튼을 누르십시오.'), Document(id='chunk-637', metadata={'brand': 'Canon', 'image_path': ['/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_622_1.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_622_2.png'], 'index': '기본 통신 설정', 'main_index': '통신 기능', 'model': 'R50', 'page': 622.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_622.jpg', 'sub_index': 'N/A'}, page_content='## 액세스 포인트 인증 방식 설정하기\n\n### 3. 인증 방식을 선택합니다.\n\n![그림 자리(인증 방식 선택 화면)]\n\n- 옵션을 선택한 다음 [OK]를 누르면 다음 화면으로 이동합니다.\n- [개방 시스템]을 선택할 경우 표시되는 [암호화 설정] 화면에서 [해제] 또는 [WEP]을 선택하십시오.\n\n## 액세스 포인트 암호 키 입력하기\n\n- 액세스 포인트에 설정된 암호 키 (비밀번호)를 입력하십시오. 설정되어 있는 암호 키에 관한 자세한 내용은 액세스 포인트의 사용 설명서를 참조하십시오.\n- 단계 4-5에 표시되는 화면은 액세스 포인트에 설정된 암호 및 인증 방식에 따라 다릅니다.\n- 단계 4-5의 화면 대신 [IP 주소 설정] 화면이 표시되면 [IP 주소 설정하기]로 이동하십시오.\n\n### 4. 키 인덱스를 선택합니다.\n\n![그림 자리(키 인덱스 선택 화면)]\n\n- 단계 3에서 [공유 키]나 [WEP]을 선택하면 [키 인덱스] 화면이 표시됩니다.\n- 액세스 포인트에 설정된 키 인덱스 번호를 선택하십시오.\n- [OK]를 선택하십시오.'), Document(id='chunk-741', metadata={'brand': 'Canon', 'image_path': [], 'index': '사용자 정의 기능 설정 항목', 'main_index': '설정', 'model': 'R50', 'page': 725.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_725.jpg', 'sub_index': 'N/A'}, page_content='```\n| 기능                      | 설정 1 | 설정 2 | 설정 3 | 설정 4 | 설정 5 | 설정 6 | 설정 7 | 설정 8 | 설정 9 | 설정 10 |\n|---------------------------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|\n| 🔋 전원 끄기              |   -    |   ○    |   ○    |   ○    |   ○    |   ○    |   ○    |   ○    |   ○    |    ○    |\n| 📷 스크린 끄기            |   -    |   ○    |   ○    |   ○    |   ○    |   ○    |   ○    |   ○    |   ○    |    ○    |\n| 📷 저소음 셔터 기능*¹     |   -    |   -    |   ○    |   ○    |   ○    |   ○    |   ○    |   ○    |   ○    |    ○    |\n| 🔍 초점/조작 링 전환      |   -    |   -    |   -    |   ○    |   ○    |   ○    |   ○    |   ○    |   ○    |    ○    |\n| 🔄 뷰파인더/화면 전환     |   -    |   -    |   -    |   -    |   ○    |   ○    |   ○    |   ○    |   ○    |    ○    |\n| 📶 Wi-Fi/블루투스 연결    |   -    |   -    |   -    |   -    |   -    |   ○    |   ○    |   ○    |   ○    |    ○    |'), Document(id='chunk-65', metadata={'brand': 'Canon', 'image_path': ['/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_70_1.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_70_2.png'], 'index': '기본 조작', 'main_index': '준비 및 기본 조작', 'model': 'R50', 'page': 70.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_70.jpg', 'sub_index': 'N/A'}, page_content='## 다이얼 사용법\n\n### (1) 버튼을 누른 후 다이얼을 돌립니다.\n\n![그림 자리(버튼을 누른 후 다이얼을 돌리는 이미지)]\n\n- **ISO** 등의 버튼을 누른 다음 다이얼을 돌리십시오. 셔터 버튼을 반누름하면 카메라가 촬영 준비 상태로 돌아갑니다.\n  - ISO 감도와 같은 설정을 조작하는 데 사용할 수 있습니다.\n\n### (2) 다이얼만 돌립니다.\n\n![그림 자리(다이얼만 돌리는 이미지)]\n\n- 스크린이나 뷰파인더를 보면서 다이얼을 돌리십시오.\n  - 셔터 스피드, 조리개 등을 설정할 때는 이 다이얼을 사용하십시오.'), Document(id='chunk-613', metadata={'brand': 'Canon', 'image_path': ['/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_598_1.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_598_2.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_598_3.png'], 'index': '프린터에 WiFi로 연결하기', 'main_index': '통신 기능', 'model': 'R50', 'page': 598.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_598.jpg', 'sub_index': 'N/A'}, page_content='### 지정한 이미지 옵션으로 인쇄하기\n\n1. **〈SET〉 버튼을 누릅니다.**\n\n   ![그림 자리(카메라 화면 예시)]\n\n2. **[인쇄 명령]을 선택합니다.**\n\n   ![그림 자리(인쇄 명령 선택 화면)]\n\n3. **인쇄 옵션을 설정합니다.**\n\n   ![그림 자리(인쇄 옵션 설정 화면)]\n\n   - 인쇄 설정 과정에 관한 내용은 [인쇄 명령 (DPOF)]을 참조하십시오.\n   - Wi-Fi 연결을 하기 전에 인쇄 명령을 완료한 경우에는 단계 4로 이동하십시오.'), Document(id='chunk-90', metadata={'brand': 'Canon', 'image_path': ['/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_95_1.png'], 'index': 'N/A', 'main_index': '베이직 존', 'model': 'R50', 'page': 95.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_95.jpg', 'sub_index': 'A+: 어시스트 기능'}, page_content='### 효과 수준과 기타 세부 요소를 선택합니다.\n\n![그림 자리(효과 설정 화면)]\n\n- `<다이얼>`로 설정한 다음 `<버튼>`을 누르십시오.\n- 설정을 초기화하려면 `<초기화 버튼>`을 누른 다음 `[OK]`를 선택하십시오.'), Document(id='chunk-618', metadata={'brand': 'Canon', 'image_path': ['/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_603_1.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_603_2.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_603_3.png'], 'index': '프린터에 WiFi로 연결하기', 'main_index': '통신 기능', 'model': 'R50', 'page': 603.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_603.jpg', 'sub_index': 'N/A'}, page_content='### 용지 타입 설정하기\n\n프린터에 준비된 용지의 종류를 선택하십시오.\n\n![그림 자리(용지 타입 설정 화면)]\n\n### 용지 레이아웃 설정하기\n\n용지의 레이아웃을 선택하십시오.\n\n![그림 자리(용지 레이아웃 설정 화면)]\n\n**주의**\n- 이미지의 화면 비율이 인쇄 용지의 가로세로 비율과 다른 경우, 이미지를 테두리 없이 인쇄하면 이미지가 상당 부분 잘려나갈 수 있습니다. 또한, 이미지가 저해상도로 인쇄될 수 있습니다.\n\n### 날짜/파일 번호 인쇄 설정하기\n\n[인쇄]를 선택하십시오.\n\n![그림 자리(날짜/파일 번호 인쇄 설정 화면)]')]
rerank_doc = [Document(id='chunk-65', metadata={'brand': 'Canon', 'image_path': ['/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_70_1.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_70_2.png'], 'index': '기본 조작', 'main_index': '준비 및 기본 조작', 'model': 'R50', 'page': 70.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_70.jpg', 'sub_index': 'N/A'}, page_content='## 다이얼 사용법\n\n### (1) 버튼을 누른 후 다이얼을 돌립니다.\n\n![그림 자리(버튼을 누른 후 다이얼을 돌리는 이미지)]\n\n- **ISO** 등의 버튼을 누른 다음 다이얼을 돌리십시오. 셔터 버튼을 반누름하면 카메라가 촬영 준비 상태로 돌아갑니다.\n  - ISO 감도와 같은 설정을 조작하는 데 사용할 수 있습니다.\n\n### (2) 다이얼만 돌립니다.\n\n![그림 자리(다이얼만 돌리는 이미지)]\n\n- 스크린이나 뷰파인더를 보면서 다이얼을 돌리십시오.\n  - 셔터 스피드, 조리개 등을 설정할 때는 이 다이얼을 사용하십시오.'), Document(id='chunk-240', metadata={'brand': 'Canon', 'image_path': ['/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_240_1.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_240_2.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_240_3.png'], 'index': '픽쳐 스타일 등록', 'main_index': '촬영 및 녹화', 'model': 'R50', 'page': 240.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_240.jpg', 'sub_index': '정지 사진 촬영'}, page_content='### 3. <SET> 버튼을 누릅니다.\n\n![그림 자리(픽처스타일 설정 화면)]\n\n- [픽처스타일]을 선택한 다음 <SET> 버튼을 누르십시오.\n\n### 4. 기본 픽처 스타일을 선택합니다.\n\n![그림 자리(기본 픽처 스타일 선택 화면)]\n\n- 기본 픽처 스타일을 선택한 다음 <SET> 버튼을 누르십시오.\n- EOS Utility (EOS 소프트웨어)로 카메라에 등록한 스타일을 조정할 때도 이 방법으로 스타일을 선택하십시오.\n\n### 5. 옵션을 선택합니다.\n\n![그림 자리(옵션 선택 화면)]\n\n- 옵션을 선택한 다음 <SET> 버튼을 누르십시오.'), Document(id='chunk-618', metadata={'brand': 'Canon', 'image_path': ['/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_603_1.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_603_2.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_603_3.png'], 'index': '프린터에 WiFi로 연결하기', 'main_index': '통신 기능', 'model': 'R50', 'page': 603.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_603.jpg', 'sub_index': 'N/A'}, page_content='### 용지 타입 설정하기\n\n프린터에 준비된 용지의 종류를 선택하십시오.\n\n![그림 자리(용지 타입 설정 화면)]\n\n### 용지 레이아웃 설정하기\n\n용지의 레이아웃을 선택하십시오.\n\n![그림 자리(용지 레이아웃 설정 화면)]\n\n**주의**\n- 이미지의 화면 비율이 인쇄 용지의 가로세로 비율과 다른 경우, 이미지를 테두리 없이 인쇄하면 이미지가 상당 부분 잘려나갈 수 있습니다. 또한, 이미지가 저해상도로 인쇄될 수 있습니다.\n\n### 날짜/파일 번호 인쇄 설정하기\n\n[인쇄]를 선택하십시오.\n\n![그림 자리(날짜/파일 번호 인쇄 설정 화면)]'), Document(id='chunk-678', metadata={'brand': 'Canon', 'image_path': [], 'index': 'N/A', 'main_index': '설정', 'model': 'R50', 'page': 662.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_662.jpg', 'sub_index': 'N/A'}, page_content='# 설정\n\n이 장에서는 설정 탭의 메뉴 설정에 관해 설명합니다.  \n제목 우측의 ★는 해당 기능이 크리에이티브 존 모드 (<P>, <Tv>, <Av>, <M>)에서만 사용 가능함을 나타냅니다.\n\n- 탭 메뉴: 설정\n- 폴더 설정\n- 파일 번호\n- 카드 포맷\n- 자동 회전\n- 동영상에 방향 정보 추가하기\n- 날짜/시간/지역\n- 언어\n- 비디오 형식\n- 촬영 모드 안내\n- 기능 안내\n- 표시음\n- 음량\n- 전원\n- 스크린 및 뷰파인더 표시\n- 스크린 밝기\n- 뷰파인더 밝기\n- 뷰파인더 색조 미세 조정\n- 메뉴 화면 확대\n- HDMI 해상도\n- 터치 제어\n- USB 연결을 위한 앱 선택\n- 카메라 설정 초기화 ★\n- 커스텀 촬영 모드 (C 모드)\n- 배터리 정보\n- 저작권 정보 ★\n- 기타 정보'), Document(id='chunk-90', metadata={'brand': 'Canon', 'image_path': ['/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_95_1.png'], 'index': 'N/A', 'main_index': '베이직 존', 'model': 'R50', 'page': 95.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_95.jpg', 'sub_index': 'A+: 어시스트 기능'}, page_content='### 효과 수준과 기타 세부 요소를 선택합니다.\n\n![그림 자리(효과 설정 화면)]\n\n- `<다이얼>`로 설정한 다음 `<버튼>`을 누르십시오.\n- 설정을 초기화하려면 `<초기화 버튼>`을 누른 다음 `[OK]`를 선택하십시오.')]

In [22]:
retrieveal_doc

[Document(id='chunk-680', metadata={'brand': 'Canon', 'image_path': ['/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_664_1.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_664_2.png'], 'index': '탭 메뉴: 설정', 'main_index': '설정', 'model': 'R50', 'page': 664.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_664.jpg', 'sub_index': 'N/A'}, page_content='## 설정 3\n\n![그림 자리(설정 3 메뉴 화면)]\n\n1. **화면/뷰파인더 표시**\n2. **화면 밝기**\n3. **뷰파인더 밝기**\n4. **뷰파인더 색조 미세조정**\n5. **메뉴 화면 확대**\n6. **HDMI 해상도**\n\n## 설정 4\n\n![그림 자리(설정 4 메뉴 화면)]\n\n1. **터치 제어**\n2. **USB 연결 앱 선택**'),
 Document(id='chunk-678', metadata={'brand': 'Canon', 'image_path': [], 'index': 'N/A', 'main_index': '설정', 'model': 'R50', 'page': 662.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~Clo

In [23]:
rerank_doc

[Document(id='chunk-65', metadata={'brand': 'Canon', 'image_path': ['/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_70_1.png', '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/image/r50/r50_page_70_2.png'], 'index': '기본 조작', 'main_index': '준비 및 기본 조작', 'model': 'R50', 'page': 70.0, 'pdf_path': '/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/parse&chunk/data/pdf/split_pdf_image/r50/r50_page_70.jpg', 'sub_index': 'N/A'}, page_content='## 다이얼 사용법\n\n### (1) 버튼을 누른 후 다이얼을 돌립니다.\n\n![그림 자리(버튼을 누른 후 다이얼을 돌리는 이미지)]\n\n- **ISO** 등의 버튼을 누른 다음 다이얼을 돌리십시오. 셔터 버튼을 반누름하면 카메라가 촬영 준비 상태로 돌아갑니다.\n  - ISO 감도와 같은 설정을 조작하는 데 사용할 수 있습니다.\n\n### (2) 다이얼만 돌립니다.\n\n![그림 자리(다이얼만 돌리는 이미지)]\n\n- 스크린이나 뷰파인더를 보면서 다이얼을 돌리십시오.\n  - 셔터 스피드, 조리개 등을 설정할 때는 이 다이얼을 사용하십시오.'),
 Document(id='chunk-240', metadata={'brand': 'Canon', 'image_path': ['/Users/yoeun/Library/Mobile Do

### 추후 참고

In [ ]:
# 추후 사용
# retrieval_grader.py
# 문서가 실제로 질몬과 관련이 있는지 판단하는 내용

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

llm = ChatOpenAI(temperature=0)


class GradeDocuments(BaseModel):
    """Binary score for relevance score on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


structured_llm_grader = llm.with_structured_output(GradeDocuments)

system = """You are a grade accessing relevance of a retrieved document to a user question. \n
If the document contains keywors(s) or semantic meaning related to the question, grade it as relevant. \n
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader


In [ ]:
## grade_documents.py
# 모든 문서를평가하는 노드

from typing import Any, Dict

from graphs.chains.retrieval_grader import retrieval_grader
from graphs.state import GraphState


def grade_documents(state: GraphState) -> Dict[str, Any]:
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    filtered_docs = []
    web_search = False
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")

    if len(filtered_docs) == 0:
        web_search = True

    return {"documents": filtered_docs, "question": question, "web_search": web_search}


In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from rank_bm25 import BM25Okapi
from kiwipiepy import Kiwi

# ✅ 1. Kiwi Tokenizer (BM25용)
kiwi = Kiwi()

def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]  # 형태소 추출

# ✅ 2. Pinecone 설정
embedding_model = OpenAIEmbeddings()
vectorstore = Pinecone(index, embedding_model)

# ✅ 3. 문서 데이터 (BM25용)
documents = [
    "머신러닝은 인공지능의 한 분야입니다.",
    "딥러닝은 신경망을 활용하는 머신러닝 기법입니다.",
    "자연어 처리(NLP)는 컴퓨터가 인간의 언어를 이해하는 기술입니다.",
    "강화 학습은 게임 AI와 로보틱스에 사용됩니다.",
]

# ✅ 4. BM25 모델 생성
tokenized_docs = [kiwi_tokenize(doc) for doc in documents]
bm25 = BM25Okapi(tokenized_docs)

# ✅ 5. Hybrid Search 함수
def hybrid_search(query, top_k=3, bm25_weight=0.5, vector_weight=0.5):
    """
    Hybrid Search (BM25 + Vector Search)
    """
    # 1️⃣ BM25 검색 실행
    tokenized_query = kiwi_tokenize(query)
    bm25_scores = bm25.get_scores(tokenized_query)  # BM25 점수 계산
    bm25_results = [
        {"id": f"doc_{i}", "bm25_score": bm25_scores[i], "text": documents[i]}
        for i in range(len(documents))
    ]

    # 2️⃣ Pinecone Vector Search 실행
    vector_results = vectorstore.similarity_search_with_score(query, k=top_k)

    # 3️⃣ Hybrid Score 결합
    combined_results = {}

    # BM25 결과 추가
    for doc in bm25_results:
        doc_id = doc["id"]
        combined_results[doc_id] = {
            "bm25_score": doc["bm25_score"],
            "vector_score": 0,
            "text": doc["text"]
        }

    # Pinecone 결과 추가
    for doc, vector_score in vector_results:
        doc_id = doc.metadata.get("id", "unknown")  # ID가 없을 경우 대비
        if doc_id in combined_results:
            combined_results[doc_id]["vector_score"] = vector_score
        else:
            combined_results[doc_id] = {
                "bm25_score": 0,
                "vector_score": vector_score,
                "text": doc.page_content
            }

    # 4️⃣ 최종 Hybrid Score 계산 (가중 평균)
    for doc_id in combined_results:
        combined_results[doc_id]["final_score"] = (
            bm25_weight * combined_results[doc_id]["bm25_score"]
            + vector_weight * combined_results[doc_id]["vector_score"]
        )

    # 5️⃣ 최종 정렬 및 결과 반환
    sorted_results = sorted(
        combined_results.items(),
        key=lambda x: x[1]["final_score"],
        reverse=True
    )

    print("\n🔎 Hybrid Search Results:")
    for rank, (doc_id, data) in enumerate(sorted_results[:top_k], start=1):
        print(f"{rank}. [{doc_id}] Score: {data['final_score']:.4f} | Text: {data['text']}")

    return sorted_results[:top_k]

# ✅ 6. Hybrid Search 실행
query_text = "인공지능과 머신러닝"
hybrid_search(query_text)
